In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [4]:
spark = SparkSession.builder.master('local[*]').appName('Spark SQL').getOrCreate()

In [5]:
passengers_df = spark.read.csv('lax_passengers_header.csv', header = True, inferSchema = True)

In [6]:
passengers_df.show(5)

+--------------------+--------------------+-----------------+-----------------+----------------------+---------------+
|     DataExtractDate|        ReportPeriod|         Terminal|Arrival_Departure|Domestic_International|Passenger_Count|
+--------------------+--------------------+-----------------+-----------------+----------------------+---------------+
|05/01/2014 12:00:...|01/01/2006 12:00:...|Imperial Terminal|          Arrival|              Domestic|            490|
|05/01/2014 12:00:...|01/01/2006 12:00:...|Imperial Terminal|        Departure|              Domestic|            498|
|05/01/2014 12:00:...|01/01/2006 12:00:...|   Misc. Terminal|          Arrival|              Domestic|            753|
|05/01/2014 12:00:...|01/01/2006 12:00:...|   Misc. Terminal|        Departure|              Domestic|            688|
|05/01/2014 12:00:...|01/01/2006 12:00:...|       Terminal 1|          Arrival|              Domestic|         401535|
+--------------------+--------------------+-----

In [7]:
passengers = passengers_df.select('ReportPeriod','Terminal','Passenger_Count')

In [8]:
passengers.show(3)

+--------------------+-----------------+---------------+
|        ReportPeriod|         Terminal|Passenger_Count|
+--------------------+-----------------+---------------+
|01/01/2006 12:00:...|Imperial Terminal|            490|
|01/01/2006 12:00:...|Imperial Terminal|            498|
|01/01/2006 12:00:...|   Misc. Terminal|            753|
+--------------------+-----------------+---------------+
only showing top 3 rows



In [9]:
approved_terminals = [f'Terminal {i}'for i in range(1,9)] + ['Tom Bradley International Terminal']

In [10]:
passengers = passengers.filter(passengers['Terminal'].isin(approved_terminals))

In [11]:
passengers.show(5)

+--------------------+----------+---------------+
|        ReportPeriod|  Terminal|Passenger_Count|
+--------------------+----------+---------------+
|01/01/2006 12:00:...|Terminal 1|         401535|
|01/01/2006 12:00:...|Terminal 1|         389745|
|01/01/2006 12:00:...|Terminal 1|            561|
|01/01/2006 12:00:...|Terminal 2|          98991|
|01/01/2006 12:00:...|Terminal 2|         163067|
+--------------------+----------+---------------+
only showing top 5 rows



In [12]:
passengers = passengers.select(split(passengers.ReportPeriod, '/')[0].alias('Month'),
                  split(passengers.ReportPeriod, '/')[2].alias('Year'),
                  'Terminal','Passenger_Count')

In [13]:
passengers.show(5)

+-----+----------------+----------+---------------+
|Month|            Year|  Terminal|Passenger_Count|
+-----+----------------+----------+---------------+
|   01|2006 12:00:00 AM|Terminal 1|         401535|
|   01|2006 12:00:00 AM|Terminal 1|         389745|
|   01|2006 12:00:00 AM|Terminal 1|            561|
|   01|2006 12:00:00 AM|Terminal 2|          98991|
|   01|2006 12:00:00 AM|Terminal 2|         163067|
+-----+----------------+----------+---------------+
only showing top 5 rows



In [14]:
clean_passengers = passengers.select('Month',split(passengers.Year, ' ')[0].alias('Year'),'Terminal','Passenger_Count')

In [21]:
clean_passengers.show(2)

+-----+----+----------+---------------+
|Month|Year|  Terminal|Passenger_Count|
+-----+----+----------+---------------+
|   01|2006|Terminal 1|         401535|
|   01|2006|Terminal 1|         389745|
+-----+----+----------+---------------+
only showing top 2 rows



In [22]:
group_passengers = clean_passengers.groupBy('Month','Year').agg(sum('Passenger_Count').alias('Passenger_Count'))

In [23]:
group_passengers.show(2)

+-----+----+---------------+
|Month|Year|Passenger_Count|
+-----+----+---------------+
|   04|2012|        5160973|
|   03|2014|        5622137|
+-----+----+---------------+
only showing top 2 rows



In [32]:
group_passengers.filter(group_passengers.Passenger_Count > 5000000).sort(asc('Month')).show(truncate = False)

+-----+----+---------------+
|Month|Year|Passenger_Count|
+-----+----+---------------+
|01   |2017|6213892        |
|01   |2014|5263473        |
|01   |2016|5869759        |
|01   |2015|5339983        |
|02   |2016|5280381        |
|02   |2017|5387511        |
|03   |2007|5240144        |
|03   |2017|6525366        |
|03   |2012|5252277        |
|03   |2006|5088556        |
|03   |2008|5232233        |
|03   |2013|5385531        |
|03   |2014|5622137        |
|03   |2015|5916774        |
|03   |2016|6277897        |
|04   |2007|5172120        |
|04   |2006|5085946        |
|04   |2012|5160973        |
|04   |2016|6109205        |
|04   |2015|5746373        |
+-----+----+---------------+
only showing top 20 rows

